In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader

# --- Load load data from CSV ---
df = pd.read_csv('synthetic_load.csv')

# Ensure the 'load' column exists
if 'load' not in df.columns:
    raise ValueError("The CSV must contain a 'load' column.")

data = df[['load']].values

# --- Normalize data ---
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(data)

# --- Create sequences (past seq_length → next value) ---
def create_sequences(data, seq_length):
    xs, ys = [], []
    for i in range(len(data) - seq_length):
        xs.append(data[i:i + seq_length])
        ys.append(data[i + seq_length])
    return np.array(xs), np.array(ys)

seq_length = 24  # Use past 24 hours to predict next hour
X, y = create_sequences(scaled_data, seq_length)

# --- Split (80/20) ---
train_size = int(0.8 * len(X))
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# --- Convert to PyTorch tensors ---
X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32)

# --- DataLoader for batching ---
train_dataset = TensorDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# --- Define LSTM Model ---
class LSTMModel(nn.Module):
    def __init__(self, input_dim=1, hidden_dim=50, num_layers=1, output_dim=1):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim)
        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])  # Output at last timestep
        return out

model = LSTMModel()

# --- Loss and optimizer ---
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

# --- Training loop ---
epochs = 30
for epoch in range(epochs):
    model.train()
    for batch_x, batch_y in train_loader:
        optimizer.zero_grad()
        outputs = model(batch_x)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()
    if epoch  % 10 == 0:
        print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}')

# --- Evaluation ---
model.eval()
with torch.no_grad():
    predicted = model(X_test)

# Inverse transform predictions and actuals
predicted = scaler.inverse_transform(predicted.numpy())
y_test_inv = scaler.inverse_transform(y_test.numpy())

# --- Metrics ---
rmse = np.sqrt(mean_squared_error(y_test_inv, predicted))
rms = np.sqrt(np.mean(np.square(y_test_inv)))  # RMS of actual values
mae = mean_absolute_error(y_test_inv, predicted)
mean_avg = np.mean(y_test_inv)  # Mean of actual values

# --- Print results ---
print(f'RMSE: {rmse:.2f}')
print(f'RMS (actual values): {rms:.2f}')
print(f'MAE: {mae:.2f}')
print(f'Mean Average (actual values): {mean_avg:.2f}')

# --- Save model ---
torch.save(model.state_dict(), 'load_lstm.pth')
print(" Model trained and saved as 'load_lstm.pth'")


Epoch [1/30], Loss: 0.1073
Epoch [11/30], Loss: 0.0008
Epoch [21/30], Loss: 0.0011
RMSE: 90.49
RMS (actual values): 3371.61
MAE: 73.26
Mean Average (actual values): 3262.91
 Model trained and saved as 'load_lstm.pth'
